# Implicit Recommendation with X-wines Dataset
[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/biodatlab/xlab-recommendation/blob/main/solution_notebooks/HW_01_X-wines.ipynb) 

Use [X-wines](https://github.com/rogerioxavier/X-Wines/blob/main/Dataset/last/XWines_Test_1K_ratings.csv) dataset for implicit recommendation



In [ ]:
!pip install implicit
!pip install gradio

#### Download data

In [ ]:
!wget https://github.com/rogerioxavier/X-Wines/raw/main/Dataset/last/XWines_Test_100_wines_1K_ratings.zip
!unzip XWines_Test_100_wines_1K_ratings.zip

In [ ]:
import pandas as pd
rating_df = pd.read_csv("XWines_Test_1K_ratings.csv")
wine_df = pd.read_csv("XWines_Test_100_wines.csv")
# wine_df.head()
# rating_df.head()

#### Create an item-user sparse matrix

In [ ]:
from scipy.sparse import csr_matrix
rating_df["WineID"] = rating_df["WineID"].astype("category")
rating_df["UserID"] = rating_df["UserID"].astype("category")

# Create an item-user relationship for bm25_weight.
wine_user_rating = csr_matrix((rating_df["Rating"].astype(float), 
                   (rating_df["WineID"].cat.codes, 
                    rating_df["UserID"].cat.codes)))

print(wine_user_rating)


#### Convert explicit data into implicit data

In [ ]:
import numpy as np 
min_rating = 4.0
wine_user_rating.data[wine_user_rating.data < min_rating] = 0
wine_user_rating.eliminate_zeros()
wine_user_rating.data = np.ones(len(wine_user_rating.data))
print(wine_user_rating)

#### Weight data with bm25_weight

In [ ]:
from implicit.nearest_neighbours import bm25_weight
wine_user_rating = (bm25_weight(wine_user_rating, B=0.9)*5).tocsr()
print(wine_user_rating)

#### Create an alternating least square model

In [ ]:
from implicit.als import AlternatingLeastSquares

model = AlternatingLeastSquares()
# Implicit expect user-item (userID-wineID)
user_wine = wine_user_rating.T.tocsr()

model.fit(user_wine)

#### Show the result

In [ ]:
# userid = 100
# # Get top 10 wines recommendation for a single user
# ids, scores = model.recommend(userid, user_wine[userid], N=10, filter_already_liked_items=False)

# # Create categories for the "WineID" column in the rating_df as categorical.
# wine_id_categories = rating_df["WineID"].cat.categories
# wine_recommended_df = wine_df.iloc[ids]
# wine_recommended_df = wine_recommended_df[["WineID","WineName"]]

# recommend_df = pd.DataFrame({"WineID": wine_id_categories[ids], "score": scores, "already_liked": np.in1d(ids, user_wine[userid].indices),})
# recommend_df = wine_recommended_df.merge(recommend_df, how="inner", on="WineID")

In [ ]:
import gradio as gr

def wine_recommend(userid):
    userid = int(userid)
    ids, scores = model.recommend(userid, user_wine[userid], N=10, filter_already_liked_items=False)

    # Create categories for the "WineID" column in the rating_df as categorical.
    wine_id_categories = rating_df["WineID"].cat.categories
    wine_recommended_df = wine_df.iloc[ids]
    wine_recommended_df = wine_recommended_df[["WineID","WineName"]]

    df = pd.DataFrame({"WineID": wine_id_categories[ids], "score": scores, "already_liked": np.in1d(ids, user_wine[userid].indices),})
    df = wine_recommended_df.merge(df, how="inner", on="WineID")
    return df

demo = gr.Interface(
    fn=wine_recommend,
    inputs="text",
    outputs="dataframe",
)
demo.launch(share=True)

In [ ]:
# itemid = 73
# ids, scores = model.similar_items(itemid)
# pd.DataFrame({"wineID":wine_id_categories[ids], "score": scores})

In [ ]:
import gradio as gr

def wine_similarity(itemid):
    itemid = int(itemid)
    ids, scores = model.similar_items(itemid)

    # Create categories for the "WineID" column in the rating_df as categorical.
    wine_id_categories = rating_df["WineID"].cat.categories
    wine_recommended_df = wine_df.iloc[ids]
    wine_recommended_df = wine_recommended_df[["WineID","WineName"]]

    df = pd.DataFrame({"wineID":wine_id_categories[ids], "score": scores})
    df = wine_recommended_df.merge(df, how="inner", on="WineID")
    return df

demo = gr.Interface(
    fn=wine_similarity,
    inputs="text",
    outputs="dataframe",
)
demo.launch(share=True)